In [1]:
# import all the necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# load and read dataset
dataset = pd.read_csv(r"C:\Users\Arfan Shah\Downloads\fraud test.csv")
dataset.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Arfan Shah\\Downloads\\fraud test.csv'

In [24]:
# remove first column and first and second name
dataset = dataset.drop(columns = ['Unnamed: 0', 'first', 'last', 'trans_date_trans_time', 'trans_num', 'merchant', 'street', 'state', 'dob'], axis = 1)
dataset.head()

,cc_num,category,amt,gender,city,zip,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud
0,2.291160e+15,personal_care,2.86,M,Columbia,29209,33.9659,-80.9355,333497,Mechanical engineer,1371816865,33.986391,-81.200714,0
1,3.573030e+15,personal_care,29.84,F,Altonah,84002,40.3207,-110.4360,302,"Sales professional, IT",1371816873,39.450498,-109.960431,0
2,3.598220e+15,health_fitness,41.28,F,Bellmore,11710,40.6729,-73.5365,34496,"Librarian, public",1371816893,40.495810,-74.196111,0
3,3.591920e+15,misc_pos,60.05,M,Titusville,32780,28.5697,-80.8191,54767,Set designer,1371816915,28.812398,-80.883061,0
4,3.526830e+15,travel,3.19,M,Falmouth,49632,44.2529,-85.0170,1126,Furniture designer,1371816917,44.959148,-85.884734,0


In [25]:
# find unique values in merchant
len(dataset.gender.value_counts())

2

In [26]:
# unique values in category
len(dataset.category.value_counts())

14

In [27]:
# find categories in job
len(dataset.job.value_counts())

478

In [28]:
# unique values in city
len(dataset.city.value_counts())

849

In [29]:
# now remove city and job columns as well
dataset = dataset.drop(columns = ['job', 'city', 'category'], axis = 1)
dataset.head()

,cc_num,amt,gender,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
0,2.291160e+15,2.86,M,29209,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714,0
1,3.573030e+15,29.84,F,84002,40.3207,-110.4360,302,1371816873,39.450498,-109.960431,0
2,3.598220e+15,41.28,F,11710,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111,0
3,3.591920e+15,60.05,M,32780,28.5697,-80.8191,54767,1371816915,28.812398,-80.883061,0
4,3.526830e+15,3.19,M,49632,44.2529,-85.0170,1126,1371816917,44.959148,-85.884734,0


In [30]:
# apply labelencoding on gender column
encoder = LabelEncoder()

dataset.gender = encoder.fit_transform(dataset['gender'])

dataset.head()

,cc_num,amt,gender,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
0,2.291160e+15,2.86,1,29209,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714,0
1,3.573030e+15,29.84,0,84002,40.3207,-110.4360,302,1371816873,39.450498,-109.960431,0
2,3.598220e+15,41.28,0,11710,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111,0
3,3.591920e+15,60.05,1,32780,28.5697,-80.8191,54767,1371816915,28.812398,-80.883061,0
4,3.526830e+15,3.19,1,49632,44.2529,-85.0170,1126,1371816917,44.959148,-85.884734,0


In [31]:
# find null values in dataset
dataset.isnull().sum()

cc_num        0
amt           0
gender        0
zip           0
lat           0
long          0
city_pop      0
unix_time     0
merch_lat     0
merch_long    0
is_fraud      0
dtype: int64

In [32]:
# find datatypes of all columns in dataset
dataset.dtypes

cc_num        float64
amt           float64
gender          int32
zip             int64
lat           float64
long          float64
city_pop        int64
unix_time       int64
merch_lat     float64
merch_long    float64
is_fraud        int64
dtype: object

In [33]:
# find distribution of target variable
dataset.is_fraud.value_counts()

is_fraud
0    553574
1      2145
Name: count, dtype: int64

In [34]:
# find correlation of all independent variables with respect to dependent variable
dataset.corr()['is_fraud']

cc_num       -0.001540
amt           0.182267
gender        0.000748
zip          -0.002271
lat           0.005863
long         -0.000972
city_pop     -0.004910
unix_time    -0.013066
merch_lat     0.005812
merch_long   -0.001060
is_fraud      1.000000
Name: is_fraud, dtype: float64

In [35]:
# length of dataset
len(dataset)

555719

In [38]:
dataset.shape

(555719, 11)

In [58]:
# sample_data
sample_data = dataset.sample(300)
sample_data.shape

(1000, 11)

In [59]:
# distribute dataset into independent and dependent sets
x = sample_data.drop(columns = ['is_fraud'])
y = sample_data['is_fraud']
x.head()

,cc_num,amt,gender,zip,lat,long,city_pop,unix_time,merch_lat,merch_long
523727,4.839040e+15,130.44,0,52563,41.1826,-92.3097,1583,1387923167,40.956835,-91.550113
307864,6.011350e+15,346.53,1,35229,33.4629,-86.7904,493806,1381880629,34.202985,-86.686197
477573,6.012000e+15,278.63,1,7640,40.9918,-73.9800,4664,1387039828,41.662418,-74.898339
78749,4.570640e+15,192.65,0,32725,28.8989,-81.2473,88735,1374198225,28.759123,-80.343270
177580,3.568740e+15,52.07,1,50073,41.8114,-93.4855,1195,1377213589,41.129116,-93.449697


In [60]:
# split dataset into training and testing

train_size = int(len(x)*0.85)

x_train = x.iloc[ : train_size, : ]
x_test = x.iloc[train_size : , : ]
print(x_train.shape)
print(x_test.shape)

# dependent variable
y_train = y[ : train_size]
y_test = y[train_size : ]

print(y_train.shape)
print(y_test.shape)

(850, 10)
(150, 10)
(850,)
(150,)


In [61]:
y_train.value_counts()

is_fraud
0    848
1      2
Name: count, dtype: int64

In [ ]:
# now apply gridsearch to find best parameters svm
best_grid = GridSearchCV(
    SVC(gamma = 'auto'),
    {
        'C': [1, 10, 20],
        'kernel': ['rbf', 'linear']
    },
    cv = 5,
    return_train_score = False
)

best_grid.fit(x_train, y_train)
best_grid.cv_results_

C:\Users\Arfan Shah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
# now apply randomisedsearchcv
best_rand = RandomizedSearchCV(
    SVC(gamma = 'auto'),
    {
        'C': [1, 10, 20],
        'kernel': ['rbf', 'linear']
    },
    cv = 5,
    return_train_score = False,
    n_iter = 2
)

In [ ]:
# now write code to select a specific model and then parameters inside
model_params = {
    'svm': {
        'model': SVC(gamma = 'auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5 ,10]
        }
    },
    'logistic_regression': LogisticRegression(),
    'params': {
        'C': [1, 5, 10]
    }
}

In [ ]:
# find scores for the best model and best parameters for selected model
scores = []

for model_name, model_params in model_params.items():
    grid = GridSearchCV(model_params['model'], model_params['params'], cv = 5, return_train_score = False)
    grid.fit(x_train, y_train)

    scores.append({
        'model': model_name,
        'best_score': grid.best_score_,
        'best_params': grid.best_params_
    })